##### 对所有静态分析结果进行统计分析

In [13]:
import os
import json
import pprint

static_res_dir = '/root/minidroid/result/sensi_apis_trigger_path'
pp_dir = '/root/minidroid/dataset/privacy_policy'
all_res = {}
for file in os.listdir(static_res_dir):
    with open(os.path.join(static_res_dir, file), 'r') as fp:
        data = json.load(fp)
    all_res[file.replace('.json', '')] = data

miniset = set()
sensi_apis_count = {}
reachable_sensi_apis_count = {}
unreachable_sensi_apis_count = {}

for miniapp in all_res.keys():
    res = all_res[miniapp]
    for page in res.keys():
        apis = res[page]
        for api in apis.keys():
            if api not in sensi_apis_count.keys():
                sensi_apis_count[api] = 1
            else:
                sensi_apis_count[api] += 1
            if apis[api] is None:
                if api not in unreachable_sensi_apis_count.keys():
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] = 1
                else:
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] += 1
            else:
                if api not in reachable_sensi_apis_count.keys():
                    reachable_sensi_apis_count[api] = 1
                else:
                    reachable_sensi_apis_count[api] += 1

# pprint.pprint(sensi_apis_count)
# pprint.pprint(reachable_sensi_apis_count)
# pprint.pprint(unreachable_sensi_apis_count)
print(len(miniset))

# counts = 0
# for count in unreachable_sensi_apis_count.values():
#     counts += count
# print(counts)

3316


##### 对所有静态分析结果进行一致性分析

In [2]:
import config as config

class ConsistencyAnalyzer():
    def __init__(self, appid, sensi_apis, pp_guidelines):
        self.appid = appid
        self.sensi_apis = sensi_apis
        self.pp_guidelines = pp_guidelines
        self.reachable_sensi_apis = set()
        self.unrechable_sensi_apis = set()
        self.reachable_scopes = set()
        self.pp_scopes = self.privacy_policy_to_scopes()
        self.set_reachable_sensi_apis()
        self.res = {
            'appid': self.appid,
            'app_nickname': self.pp_guidelines['app_nickname'],
            'reachable_sensi_apis': list(self.reachable_sensi_apis),
            'unreachable_sensi_apis': list(self.unrechable_sensi_apis),
            'pp_scopes': list(self.pp_scopes),
            'redundant_scopes': [],
            'missing_scopes': []
        }
        self.consistency_analysis()
    
    def set_reachable_sensi_apis(self):
        for page in self.sensi_apis.keys():
            for sensi_api in self.sensi_apis[page].keys():
                if self.sensi_apis[page][sensi_api] == None:
                    self.unrechable_sensi_apis.add(sensi_api)
                else:
                    self.reachable_sensi_apis.add(sensi_api)
                    self.reachable_scopes.add(config.SENSITIVE_API[sensi_api])

    def privacy_policy_to_scopes(self):
        pp_scopes = set()
        for desc in self.pp_guidelines['privacy_detail_list']['item']:
            if '开发者将在获取你的明示同意后' in desc:
                scope = desc.split('，')[-1]
                pp_scopes.add(scope)
            elif '用于' in desc:
                scope = desc.split('，')[0].replace('开发者', '')
                pp_scopes.add(scope)
        return pp_scopes
    
    def consistency_analysis(self):
        redundant_scopes = list(self.pp_scopes - self.reachable_scopes)
        missing_scopes = list(self.reachable_scopes - self.pp_scopes)
        if len(redundant_scopes):
            self.res['redundant_scopes'] = redundant_scopes
        else:
            self.res['redundant_scopes'] = None
        if len(missing_scopes):
            self.res['missing_scopes'] = missing_scopes
        else:
            self.res['missing_scopes'] = None


consistency_res = []
for file in os.listdir(static_res_dir):
    try:
        appid = file.replace('.json', '')
        with open(os.path.join(static_res_dir, file), 'r') as fp:
            sensi_apis = json.load(fp=fp)
        with open(os.path.join(pp_dir, file), 'r') as fp:
            pp_guidelines = json.load(fp=fp)
        analyzer = ConsistencyAnalyzer(appid=appid, sensi_apis=sensi_apis, pp_guidelines=pp_guidelines)
        res = analyzer.res
        if res['redundant_scopes'] != None:
            consistency_res.append(res)
    except Exception as e:
        pass

with open('/root/minidroid/result/redundant_scopes_res.json', 'w') as fp:
    json.dump(consistency_res, fp=fp, indent=4, ensure_ascii=False)
    

##### 对一致性分析结果进行统计分析

In [29]:
with open('/root/minidroid/result/redundant_scopes_res.json', 'r') as fp:
    data = json.load(fp=fp)

redundant_scopes = {}
missing_scopes = {}

redundant_scope_miniapps = {}
missing_scope_miniapps = {}

miniapp_redundant_count = {}
miniapp_missing_count = {}


for miniapp in data:
    if miniapp['redundant_scopes'] != None:
        if len(miniapp['redundant_scopes']) not in miniapp_redundant_count.keys():
            miniapp_redundant_count[len(miniapp['redundant_scopes'])] = set()
            miniapp_redundant_count[len(miniapp['redundant_scopes'])].add(miniapp['appid'])
        else:
            miniapp_redundant_count[len(miniapp['redundant_scopes'])].add(miniapp['appid'])
        for scope in miniapp['redundant_scopes']:
            if scope not in redundant_scopes.keys():
                redundant_scopes[scope] = 1
                redundant_scope_miniapps[scope] = [miniapp['appid']]
            else:
                redundant_scopes[scope] += 1
                redundant_scope_miniapps[scope].append(miniapp['appid'])
    if miniapp['missing_scopes'] != None:
        if len(miniapp['missing_scopes']) not in miniapp_missing_count.keys():
            miniapp_missing_count[len(miniapp['missing_scopes'])] = set()
            miniapp_missing_count[len(miniapp['missing_scopes'])].add(miniapp['appid'])
        else:
            miniapp_redundant_count[len(miniapp['missing_scopes'])].add(miniapp['appid'])
        for scope in miniapp['missing_scopes']:
            if scope not in missing_scopes.keys():
                missing_scopes[scope] = 1
                missing_scope_miniapps[scope] = [miniapp['appid']]
            else:
                missing_scopes[scope] += 1
                missing_scope_miniapps[scope].append(miniapp['appid'])

# pprint.pprint(redundant_scopes)
# pprint.pprint(redundant_scope_miniapps)

# pprint.pprint(missing_scopes)
# pprint.pprint(missing_scope_miniapps)

pprint.pprint(miniapp_redundant_count)
pprint.pprint(miniapp_missing_count)

# missing_counts = 0
# for count in missing_scopes.values():
#     missing_counts += count
# print(missing_counts)


{1: {'wx16832daa44c1ffb7',
     'wx3baab8412ba688f0',
     'wx3bb8c4f5c8bd1f2f',
     'wx3d12c0f6af6c8ac7',
     'wx4511985ceab75366',
     'wx4571c1a2956fbb73',
     'wx45c38607d69fd2ea',
     'wx45f3a8c5ae9ea630',
     'wx46c415dad41f1638',
     'wx4745f9e5627224bb',
     'wx48b2f475337789d2',
     'wx49eb8f826a7d17fc',
     'wx49f45bf525ac3dce',
     'wx4b6befb2dd35d8e1',
     'wx4b8b86d58f94abc8',
     'wx4c0632008e420b72',
     'wx4ce9c3cff6c3c610',
     'wx4d4b99237ad0dc94',
     'wx4d54f84ac55705e9',
     'wx4db02b9d0b86b7d4',
     'wx4dee5123318ea54b',
     'wx4e0e6d441e54eb3f',
     'wx4e72351e2aeaace2',
     'wx4ed9365b78fef3a4',
     'wx4eda92dcca9ea7ea',
     'wx4f5b2bbe1002f9b1',
     'wx61faf86dd0454a15',
     'wx630fbc1f7c4085d8',
     'wx66265c1bb3930c32',
     'wx682f98c23c1b0b37',
     'wx703f51fd9c9a4c9d',
     'wx705c2ca403d5d389',
     'wx70b31abf1524d248',
     'wx71169d8380740bb1',
     'wx7133a226ca826746',
     'wx71375401f87c93cc',
     'wx7139e1add32420c4',
 